<a href="https://colab.research.google.com/github/sayanbanerjee32/keyword_search/blob/main/Semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Word vectors and SIF

In [1]:
%%capture
# To import kaggle datasets
!pip install --upgrade pip 
!pip install kaggle==1.5.6

In [2]:
# # import colab libraries
# from google.colab import files
# # import kaggle json to connect to kaggle user account to download datsets
# files.upload()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [4]:
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# download our dataset
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p 'datasets'

 19% 5.00M/25.7M [00:00<00:00, 31.6MB/s]
100% 25.7M/25.7M [00:00<00:00, 105MB/s] 


In [6]:
#unzip
!unzip datasets/imdb-dataset-of-50k-movie-reviews.zip -d datasets/

Archive:  datasets/imdb-dataset-of-50k-movie-reviews.zip
  inflating: datasets/IMDB Dataset.csv  


## Sentence BERT
https://evergreenllc2020.medium.com/semantic-search-engine-with-s-abbfb3cd9377

In [7]:
%%capture
!pip install sentence-transformers

In [8]:
import scipy
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
from sentence_transformers import SentenceTransformer
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Load data and create DF

In [10]:
import pandas as pd
df = pd.read_csv('datasets/IMDB Dataset.csv')
df.shape


(50000, 2)

In [11]:
df.head(1)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive


### Preprocess and tokenise

In [12]:
import re
def fix_text(txt):
    return re.sub(r'(<br />)+',' ',txt)
    # return clean(txt,
    #             fix_unicode=True,               # fix various unicode errors
    #             to_ascii=True,                  # transliterate to closest ASCII representation
    #             lower=True,                     # lowercase text
    #             no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
    #             no_urls=False,                  # replace all URLs with a special token
    #             no_emails=True,                # replace all email addresses with a special token
    #             no_phone_numbers=True,         # replace all phone numbers with a special token
    #             no_numbers=True,               # replace all numbers with a special token
    #             no_digits=False,                # replace all digits with a special token
    #             no_currency_symbols=True,      # replace all currency symbols with a special token
    #             no_punct=True,                 # remove punctuations
    #             replace_with_punct="<PUNCT>",          # instead of removing punctuations you may replace them
    #             replace_with_url="<URL>",
    #             replace_with_email="<EMAIL>",
    #             replace_with_phone_number="<PHONE>",
    #             replace_with_number="<NUMBER>",
    #             replace_with_digit="0",
    #             replace_with_currency_symbol="<CUR>",
    #             lang="en"                       # set to 'de' for German special handling
    #         )

In [13]:
import itertools

def flatten(it):
    return itertools.chain.from_iterable(it)

In [14]:
# nlp = spacy.load("en_core_web_sm")
# tok_text=[] # for our tokenised corpus
text = df.review.tolist()
text = [nltk.sent_tokenize(fix_text(str(i))) for i in text]
# spacy = WordTokenizer()
# toks = first(spacy([text]))
# print(coll_repr(toks, 30))
text = list(flatten(text))
len(text)

616287

In [15]:
# Each sentence is encoded as a 1-D vector with 78 columns
sentence_embeddings = model.encode(text)
print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))
# print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

Sample BERT embedding vector - length 768


In [19]:
# code adapted from 
# https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py

#query = 'bad weather' #@param {type: 'string'}
query = 'employee stop working' #@param {type: 'string'}
# query = "Security State Penitentary"
#query = 'global warming impact'
#query = 'wildfires in australia'

queries = [query]
query_embeddings = model.encode(queries)

# Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
number_top_matches = 10 #@param {type: "number"}

print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    print(type(results))

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:number_top_matches]:
        print(text[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results
<class 'list'>




Query: employee stop working

Top 5 most similar sentences in corpus:
Please, give up your day job. (Cosine Score: 0.8413)
Jane quits her job. (Cosine Score: 0.8360)
At the end, the gates to the factory are being closed. (Cosine Score: 0.8233)
The studio ditched it. (Cosine Score: 0.8083)
That's until the company of Dynocorpe sinks and closes down. (Cosine Score: 0.7978)
Since writing all stop. (Cosine Score: 0.7958)
Please cancel this show. (Cosine Score: 0.7844)
Everyone stopped skating. (Cosine Score: 0.7833)
Hasn't he just lost his job? (Cosine Score: 0.7821)
The loser is simply removed from existence. (Cosine Score: 0.7784)
